In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import stone_pose_hori
from scoop.msg import stone_pose_ver
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
rospy.init_node('scoop', anonymous=True) #initialize the node



In [3]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1610602238.405570]: Client test: Starting sending goals


In [4]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [5]:
import serial
import signal
import time

serialPort = "/dev/ttyACM0"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass
def softtip():
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1.5)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2090')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)

    
def hardtip():
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2180')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1.5)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2090')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)

In [6]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [7]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
    worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
    return [worldTstone[0]+0.01953-0.0032 +0.001073343, worldTstone[1]+0.02148-0.0035+0.0096 -0.003214275, worldTstone[2]]

In [8]:
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, Go_stone_dir_sorted, yaw_pitch_normal_set
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        Go_stone_dir_sorted = []
        yaw_pitch_normal_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z
        yaw = msg.yaw
        pitch = msg.pitch
        normal = msg.normal
        dir_depth_inner = msg.dir_depth_inner
        dir_depth_outer = msg.dir_depth_outer
        dir_depth_outer_minus_inner = []
        for i in range(len(dir_depth_inner)):
            dir_depth_outer_minus_inner.append(dir_depth_outer[i]-dir_depth_inner[i])
        dir_depth_outer_minus_inner_argsort = np.argsort(-np.array(dir_depth_outer_minus_inner)).tolist()
        dir_depth_outer_minus_inner_argsort_copy = dir_depth_outer_minus_inner_argsort[:]
        for index in dir_depth_outer_minus_inner_argsort:
            if dir_depth_outer[index]-dir_depth_inner[index]<0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
            elif dir_depth_outer[index]==0 or dir_depth_inner[index]==0:
                dir_depth_outer_minus_inner_argsort_copy.remove(index)
        dir_depth_outer_minus_inner_argsort=dir_depth_outer_minus_inner_argsort_copy[:]
        hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        Go_stone_dir_sorted.append(dir_depth_outer_minus_inner_argsort)
        yaw_pitch_normal_set.append([yaw,pitch,normal])
hori_pose_topic = '/hori_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, stone_pose_hori, hori_stone_pose_callback)

In [9]:
import json
def ver_stone_pose_callback(msg):
    global ver_pose_Go_stone_set, ver_Go_stone_number, gripper_direction_set, gripper_aperture_set, json_error
    ver_Go_stone_number = int(msg.ver_Go_stone_number)
    if ver_Go_stone_number == 0:
        ver_pose_Go_stone_set = []
        gripper_direction_set = []
        gripper_aperture_set = []
    else:
        try:
            pose_x = msg.x
            pose_y = msg.y
            pose_z = msg.z 
            box = json.loads(msg.box)
            ver_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
            box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
            #print box
            length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
            length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
            if length1 > length2:
                try:
                    if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                        temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                    else:
                        temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                    gripper_direction_set.append(temp_angle)
                except:
                    gripper_direction_set.append(0)
                gripper_aperture_set.append(length2+0.006)
            else:
                try:
                    if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                        temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                    else:
                        temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                    gripper_direction_set.append(temp_angle)
                except:
                    gripper_direction_set.append(0)
                gripper_aperture_set.append(length1+0.006)       
        except:
            json_error = True
ver_pose_topic = '/ver_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(ver_pose_topic, stone_pose_ver, ver_stone_pose_callback)

In [10]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/shoaib/Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/shoaib/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/shoaib/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [11]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [76]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

# first shorten, next translate, finally close
# l0=0.0125 l1=0.1 l2l=0.006 l2r=0.015 flength_l = 0.072   theta=60    joint_value = 78.88, -112.01, -106.91, -61.26, 133.28, -15.20
def pos_flength_adjust_left_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_l):
    theta=theta*pi/180
    current_flength_r = flength_l + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), -cos(theta), -current_left_fingertip_pos_g[0]*sin(theta)+current_left_fingertip_pos_g[1]*cos(theta)], [cos(theta), sin(theta), -current_left_fingertip_pos_g[0]*cos(theta)-current_left_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_right_fingertip_pos_g[0]], [current_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    translation_dir_dis = [-next_left_fingertip_pos_w[0], -next_left_fingertip_pos_w[1]]
    next_right_fingertip_pos_after_trans_w = [next_right_fingertip_pos_w[0]+translation_dir_dis[0], next_right_fingertip_pos_w[1]+translation_dir_dis[1]]
    right_finger_shorten_distance=abs(next_right_fingertip_pos_after_trans_w[1])/sin(theta)
    #translation_dir_dis[0] = translation_dir_dis[0]+(2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/sin(theta)-(next_right_fingertip_pos_after_trans_w[0]-right_finger_shorten_distance*cos(theta))
    return translation_dir_dis, right_finger_shorten_distance

def scoop_execute():
    go_to_safe_place()
    signal.setitimer(signal.ITIMER_REAL, 0.001)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1)
    Robotiq.goto(robotiq_client, pos=13*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    softtip()
    Hong_joint0 = radians(81.54)
    Hong_joint1 = radians(-94.03)
    Hong_joint2 = radians(-105.84)
    Hong_joint3 = radians(-74.76)
    Hong_joint4 = radians(118.75)
    Hong_joint5 = radians(-189.52)
    '''
    Hong_joint0 = radians(74.42)
    Hong_joint1 = radians(-101.22)
    Hong_joint2 = radians(-107.35)
    Hong_joint3 = radians(-74.53)
    Hong_joint4 = radians(129.36)
    Hong_joint5 = radians(-200.24)
    '''
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    rob.translate((0, 0, -0.095), acc=0.3, vel=0.8, wait=True)
    rob.translate((0, 0, -0.03), acc=0.01, vel=0.05, wait=False)   
                
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 5:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.15:
            num_large_force += 1
        if time.time()-time0>2.8: # need change
            break
    
    #rob.translate((0.005*sin(-gripper_orientation*pi/180), 0.005*cos(-gripper_orientation*pi/180), 0), acc=0.01, vel=0.01, wait=True)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    for current_finger_length in np.arange(0,184,4):
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(current_finger_length))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)        
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.3:
            break
    '''
    for k0 in range(0, 57, 4):
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(k0))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)
    '''
    #current_fingerlength_int=56   #need change
    angle_step=0.2
    for angle in np.arange(13,-4,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(160.0/0.025),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        if True:
        #if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        #else:
            #time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    go_to_safe_place()

In [78]:
scoop_execute()


In [ ]:
((0.075-0.07386302202082284)+(0.05-0.048753046314972556)+(0.025-0.022889629810456634)+(0.025-0.02520092608384635))/4.0

In [ ]:
((0.675-0.67981968371526)+(0.675-0.6792698785028867)+(0.7-0.7018858848101702)+(0.65-0.6518816535328663))/4.0

In [ ]:
Robotiq.goto(robotiq_client, pos=0.021, speed=0.000001, force=1, block=True)
#servo_pub.publish(180)
#softtip()

In [ ]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.000001, force=1, block=True)

In [ ]:
go_to_safe_place()
softtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)

In [ ]:
go_to_safe_place()
hardtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
rob.translate_tool((0.0065,0,0),acc=0.1, vel=0.2)

In [ ]:
go_to_safe_place()
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,0,0*pi/180), acc=0.1, vel=0.2, wait=True)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
#rob.movel_tool((0,0,0,0,-20*pi/180,0), acc=0.1, vel=0.2, wait=True)
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
hardtip()
#go_to_safe_place()
servo_pub.publish(0)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = 0.002
tcp2fingertip.pos.y = 0  
tcp2fingertip.pos.z = 0.356
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y+=0.001
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.1, vel=0.4, wait=True)
servo_pub.publish(100)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)


In [ ]:
rob.set_tcp((-0.02, 0, 0.342, 0, 0, 0))
rospy.sleep(0.5)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [ ]:
go_to_safe_place()
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rob.translate((0.05-pos_x+0.02,0.69-pos_y-0.02,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((-0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,-0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True)  

In [ ]:
        hardtip()
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001

        rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)     


In [79]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [ ]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.02 # need change
tcp2fingertip.pos.z = 0.342 # need change
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y+=0.001
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
#rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)

In [ ]:
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    print tcp2cam

In [ ]:
hardtip()

In [ ]:
print np.load('/home/terry/catkin_ws/src/scoop/src/collecting_data/Mon_Nov_23_19_36_07_2020/executed_action_list.npy')

In [ ]:
go_to_safe_place()

In [ ]:
softtip()
hardtip()

In [ ]:
    servo_pub.publish(180)
    rospy.sleep(1)


In [ ]:
rob.set_tcp((0, 0, 0.35947, 0, 0, 0))
rospy.sleep(1)
print rob.get_pose()
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(1)

In [ ]:
import serial
import signal
import time
serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
time0 = time.time()
def myHandler(signum, frame):
    pass
signal.signal(signal.SIGALRM, myHandler)
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)
print time.time()-time0
print '2'

In [ ]:

softtip()

In [ ]:
hori_pose_Go_stone_set[index[1]][0]>point0[0]-0.015 and hori_pose_Go_stone_set[index[1]][0]<point0[0]+0.015 and hori_pose_Go_stone_set[index[1]][1]>point0[1]-0.015 and hori_pose_Go_stone_set[index[1]][1]<point0[1]+0.015 and hori_pose_Go_stone_set[index[1]][2]<point0[2]

In [ ]:
#go_to_safe_place()
rob.set_tcp((0, 0, 0.36142, 0, 0, 0))
rospy.sleep(0.5)

move = m3d.Transform((0.8-rob.get_pose().pos.y+0.001, 0.05-rob.get_pose().pos.x-0.004, 0, 0, 0, 0))    
rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [ ]:
rob.set_tcp((0, 0, 0.36142, 0, 0, 0))
rospy.sleep(0.5)
rob.get_pose()

In [ ]:
go_to_end_place()

In [ ]:
      Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)


In [ ]:
hardtip()

In [ ]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1000')
time.sleep(1)
signal.setitimer(signal.ITIMER_REAL, 0)
for k0 in range(0, 81, 4):
    signal.setitimer(signal.ITIMER_REAL, 0.001)
    ser.write('1'+str(k0))
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.2)
print 4

In [ ]:
time0=time.time()
signal.setitimer(signal.ITIMER_REAL, 0)
for k0 in range(0, -31, -2):
    signal.setitimer(signal.ITIMER_REAL, 0.001)
    ser.write('1'+str(100+k0))
    print '1'+str(100+k0)
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)
print 4

In [ ]:
go_to_safe_place()


In [ ]:
switching_stone_coordinate_set

In [80]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out


In [17]:

serialPort = "/dev/ttyACM0"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass
signal.setitimer(signal.ITIMER_REAL, 0.01)
ser.write('2000')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)
signal.setitimer(signal.ITIMER_REAL, 0.01)
ser.write('2090')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(0.1)

In [ ]:
time0 = time.time()
rob.translate((0, 0, -0.015), acc=0.01, vel=0.05, wait=True)
print time.time()-time0

In [69]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [19]:
softtip()

In [18]:
hardtip()